In [ ]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.5.0+cpu.html


Looking in links: https://data.pyg.org/whl/torch-2.5.0+cpu.html


In [ ]:
!pip install certifi
!pip install torch_geometric

In [ ]:
import torch
import torch_cluster
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec, PNAConv
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import degree

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install --upgrade torch torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3367, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3147, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install

In [ ]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0].to(device)

Processing...
Done!


In [ ]:
# Configure Node2Vec parameters
embedding_dim = 64
walk_length = 20
context_size = 10
walks_per_node = 10
num_negative_samples = 1

node2vec = Node2Vec(
    data.edge_index,
    embedding_dim=embedding_dim,
    walk_length=walk_length,
    context_size=context_size,
    walks_per_node=walks_per_node,
    num_negative_samples=num_negative_samples,
    sparse=True
).to(device)

In [ ]:
# Train the Node2Vec embeddings
loader = node2vec.loader(batch_size=128, shuffle=True)
optimizer_n2v = torch.optim.SparseAdam(list(node2vec.parameters()), lr=0.01)

def train_node2vec():
    node2vec.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer_n2v.zero_grad()
        loss = node2vec.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer_n2v.step()
        total_loss += loss.item()
    return total_loss / len(loader)

print("Training Node2Vec embeddings...")
for epoch in range(1, 51):
    loss = train_node2vec()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

Training Node2Vec embeddings...
Epoch: 10, Loss: 1.2239
Epoch: 20, Loss: 0.9059
Epoch: 30, Loss: 0.8587
Epoch: 40, Loss: 0.8443
Epoch: 50, Loss: 0.8350


In [ ]:
# Extract the learned Node2Vec embeddings
node2vec.eval()
node_embeddings = node2vec().detach()  # [num_nodes, embedding_dim]

In [ ]:
# Compute node degrees for PNAConv
deg = degree(data.edge_index[0], data.num_nodes).to(device)

# Define aggregators and scalers for PNAConv
aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

hidden_dim = 64

In [ ]:
# Define a PNA-based GNN model that incorporates multiscale embeddings.

class MultiScaleGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(MultiScaleGNN, self).__init__()
        self.pna_conv = PNAConv(
            in_channels=in_channels,
            out_channels=hidden_channels,
            aggregators=['mean', 'max', 'min', 'std'],
            scalers=['identity', 'amplification', 'attenuation'],
            deg=degree(data.edge_index[0], num_nodes=data.num_nodes).to(device)
        )
        self.fc = nn.Linear(hidden_channels + embedding_dim, out_channels)  # Include Node2Vec embeddings

    def forward(self, x, edge_index):
        pna_out = self.pna_conv(x, edge_index)
        x_combined = torch.cat([pna_out, node_embeddings], dim=-1)
        return self.fc(x_combined)

model = MultiScaleGNN(
    in_channels=data.num_features,
    hidden_channels=128,
    out_channels=dataset.num_classes
).to(device)


In [ ]:
# Setup training components
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

x = data.x

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test():
    model.eval()
    out = model(x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs  # [train_acc, val_acc, test_acc]

In [ ]:
# Train and evaluate
best_val_acc = 0
test_acc_at_best_val = 0

print("Training PNA model with Multi-scale embeddings to improve representation richness....")
for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc_at_best_val = test_acc

    if epoch % 20 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, "
              f"Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")

print(f"\nBest Validation Accuracy: {best_val_acc:.4f}")
print(f"Test Accuracy at Best Val: {test_acc_at_best_val:.4f}")

Training PNA model with Multi-scale embeddings to improve representation richness....
Epoch: 020, Loss: 0.0003, Train: 1.0000, Val: 0.4200, Test: 0.4300
Epoch: 040, Loss: 0.0013, Train: 1.0000, Val: 0.4700, Test: 0.4910
Epoch: 060, Loss: 0.0004, Train: 1.0000, Val: 0.4940, Test: 0.5280
Epoch: 080, Loss: 0.0011, Train: 1.0000, Val: 0.5500, Test: 0.5890
Epoch: 100, Loss: 0.0023, Train: 1.0000, Val: 0.6180, Test: 0.6510
Epoch: 120, Loss: 0.0033, Train: 1.0000, Val: 0.6700, Test: 0.6850
Epoch: 140, Loss: 0.0037, Train: 1.0000, Val: 0.6920, Test: 0.7100
Epoch: 160, Loss: 0.0038, Train: 1.0000, Val: 0.6940, Test: 0.7210
Epoch: 180, Loss: 0.0037, Train: 1.0000, Val: 0.7020, Test: 0.7240
Epoch: 200, Loss: 0.0036, Train: 1.0000, Val: 0.7060, Test: 0.7250

Best Validation Accuracy: 0.7060
Test Accuracy at Best Val: 0.7210
